In [1]:
import matplotlib.pyplot as plt
import numpy as np
from IPython import embed
from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits, ascii
import h5py
from scipy.interpolate import interp2d, RegularGridInterpolator
from matplotlib.ticker import AutoMinorLocator,MaxNLocator
import astropy.table as tab
from scipy.spatial import cKDTree
from matplotlib.ticker import MultipleLocator, MaxNLocator
import spectres
from scipy.ndimage import convolve1d

#cosmological parameters
c = 3e5  # km/s
lya = 1215.67  # AA
h = 0.6774
OmegaM = 0.3089
OmegaB = 0.0486
#starting redshift of the TNG50
z_0 = 2.0020281392528516
Lbox = 35  # Mpc/h
ncell = 1000#len(np.sort(np.unique(f['ray_pos'][:,0])))
cosmo = FlatLambdaCDM(H0=100.0 * h, Om0=OmegaM, Ob0=OmegaB)
hubblez0 = cosmo.H(z_0)

#use the z in roughly the middle of the box
hubblez = cosmo.H(2.020)

dz0 = ((Lbox / h / ncell) * hubblez0 / c).value
dz = ((Lbox / h / ncell) * hubblez / c).value # dz in redshift per cell
dWL = (dz * lya)  # wl in AA per cell
dl = Lbox / ncell  # dist per cell for raw skewer in Mpc/h


In [2]:
f_SDSS = h5py.File('/data/forest/dsantos/DylanSims/Data/z2/SDSS/Random/spectra_TNG50-1_z2.0_n2000d2-rndfullbox_SDSS-BOSS_HI_combined.hdf5', 'r')
f = h5py.File('/data/forest/dsantos/DylanSims/Data/z2/KECK/Random/spectra_TNG50-1_z2.0_n2000d2-rndfullbox_KECK-HIRES-B14_HI_combined.hdf5', 'r')
wavelength_all = np.array(f['wave'])

f1 = ascii.read('/data/forest/dsantos/DylanSims/Data/z2/SubhaloInfo.csv',format='csv')

subhalo_posx = np.array(f1['Subhalo_PosX']) # Subhalo positions in x
subhalo_posy = np.array(f1['Subhalo_PosY']) # Subhalo positions in y
subhalo_posz = np.array(f1['Subhalo_PosZ']) # Subhalo positions in z
subhalo_mass = np.array(f1['Subhalo_Mass']) # Subhalo mass. To convert to physical mass you have to multiply by 1e10/H0
subhalo_radhm = np.array(f1['Subhalo_HalfMassRadius']) # Subhalo half mass radius. Twice this is the virial radius.
subhalo_z = np.array(f1['Subhalo_GasMetal']) # Subhalo gas metallicity
subhalo_vz = np.array(f1['Subhalo_PVZ']) # Subhalo peculiar velocity in z axis (km/s)
subhalo_vdisp = np.array(f1['Subhalo_VDispersion']) # Subhalo velocity dispersion (km/s)
subhalo_vmax = np.array(f1['Subhalo_VMax']) # Subhalo maximum velocity of the rotation curve (km/s)

#f = ascii.read('/data/forest/dsantos/DylanSims/Data/z2/GroupInfo.csv',format='csv')
#group_posx = np.array(f['Group_CMX']) # Group positions in x
#group_posy = np.array(f['Group_CMY']) # Group positions in y
#group_posz = np.array(f['Group_CMZ']) # Group positions in z
#group_mass = np.array(f['Group_Mass']) # Group Mass
#group_z = np.array(f['Group_Metal']) # Group Metallicity
#group_vrad = np.array(f['Group_RCrit200']) # Group virial radius
#group_subhaloid = np.array(f['Subhalo_ID']) # Central Subhalo ID

In [3]:
def wave_rebin(bins, wavelength):
    wavelength_rebin = np.zeros(int(len(wavelength)/bins))

    i = 0
    wi = 0
    while wi < len(wavelength_rebin):
        wavelength_rebin[wi] = np.sum(wavelength[i:i+bins])/bins
        wi += 1
        i += bins

    lya = np.where((wavelength_rebin > 3640) & (wavelength_rebin < 3652)) # Lyman alpha at z=2. is 3647.01 Angstroms  
    
    wave1 = wavelength_rebin[lya][-1]
    wave2 = wavelength_rebin[lya][-2]
    
    c = 2.99792e5 #km/s
    wave_lya = 1215.670 #Angstrom
    z1 = wave1/wave_lya - 1
    z2 = wave2/wave_lya - 1
    dz = (wave1 - wave2)/wave_lya
    deltav = c * dz/(1+z1)
    
    return wavelength_rebin, deltav



In [4]:
#DLA_list ='/data/forest/dsantos/DylanSims/Data/z2/KECK/Random/Old_SpecBins/DLA_SBLA_HighRes/DLA_AllBins_Index.fits'
DLA_list ='/data/forest/dsantos/DylanSims/Data/z2/SDSS/Random/DLA_SBLA/DLA_AllBins_Index.fits'
f_DLA=tab.Table.read(DLA_list, hdu=1)

#mark DLA index
all_indices = np.arange(len(f['flux']))
not_DLA = np.isin(all_indices, f_DLA[0][0], invert=True)
ind_not_DLA = all_indices[not_DLA]
ind_DLA = all_indices[~not_DLA]

In [5]:
#convert from z position to wavelength
z_halo = subhalo_posz/(dl*1000)  * dz+ (z_0)
#wl_halo = (1+z_halo )* lya
wl_halo = (1+z_halo + (1+z_halo)*subhalo_vz/c )* lya

#wavelength of the halos corresponding to their real space location. (vz=0)
wl_halo_v0 = (1+z_halo )* lya

# location of a mass=12 halo
mass_filter_0 = (np.log10(subhalo_mass*1e10/0.6774) > 11.45) & (np.log10(subhalo_mass*1e10/0.6774) < 11.55)
np.sum(mass_filter_0)


np.int64(111)

In [ ]:
for i_halo in [4, 10, 20, 45, 46, 48, 59, 69, 71, 73, 76, 85, 87, 91, 100, 102, 110]:#np.arange(0,10): #, 4, 10, 20, 45, 46, 48, 59, 69, 71, 73, 76, 85, 87, 91, 100, 102, 110]:#np.arange(0,np.sum(mass_filter_0)):
    
    print("halo ",i_halo)
    
    halo_number=i_halo    
    
    halo_0_x = subhalo_posx[mass_filter_0][halo_number]
    wl_halo_0 = ( (z_0) + subhalo_posz[mass_filter_0][halo_number]/(dl*1000)  * dz
             + (1+z_halo[mass_filter_0][halo_number])*subhalo_vz[mass_filter_0][halo_number]/c +1 )* lya
    
    halo_0_r = 2*subhalo_radhm[mass_filter_0][halo_number]
    
    halo_0_dwl= (1+z_halo[mass_filter_0][halo_number])*subhalo_vmax[mass_filter_0][halo_number]/c * lya
    
    #the width of slice in kpc/h
    y_pos = subhalo_posy[mass_filter_0][halo_number]
    
    dy=35
    
    # extendthe bourandary a little bit 
    wl_min = (1+z_0)*lya-5
    wl_max = (1+z_0 + 1000*dz)*lya+5
    
    y_ind = np.where((f['ray_pos'][:,1]>=y_pos-dy/2)&(f['ray_pos'][:,1]<=y_pos+dy/2))[0]
    y_ind_not_DLA = np.intersect1d(y_ind, ind_not_DLA)
    
    #label the DLA los in the panel
    y_ind_DLA = np.intersect1d(y_ind, ind_DLA)
    x_los_DLA = f['ray_pos'][y_ind_DLA][:,0]
    
    wl_ind = np.where((wavelength_all<wl_max) & (wavelength_all>wl_min))[0]
    
    ray_z = wavelength_all[wl_ind]
    
    vel_arr_z = (ray_z-(ray_z[0]+ray_z[-1])/2)/wl_halo_0 * c # in km/s
    d_arr_z = np.arange(len(vel_arr_z)) * dl *1000 # in kpc/h
    
    #los that go through the halo
    x_ind_halo0_los = \
    np.where((f['ray_pos'][y_ind_not_DLA][:, 0] >= halo_0_x - halo_0_r) & (f['ray_pos'][y_ind_not_DLA][:, 0] <= halo_0_x + halo_0_r))[0]
    x_halo0_los = f['ray_pos'][y_ind_not_DLA][:, 0][x_ind_halo0_los]
    
    wl_halo_0_max = wl_halo_0 + (1+z_0+subhalo_posz[mass_filter_0][halo_number]/(dl*1000) * dz)*subhalo_vmax[mass_filter_0][halo_number]/c * lya
    wl_halo_0_min = wl_halo_0 - (1+z_0+subhalo_posz[mass_filter_0][halo_number]/(dl*1000) * dz)*subhalo_vmax[mass_filter_0][halo_number]/c * lya
    
    #condition if for all subhalos in the plane of the slice
    #condition0 = (subhalo_posy - y_pos)**2  <= (35/2)**2
    condition1 = (subhalo_posy - y_pos)**2  <= (2*subhalo_radhm)**2
    condition2 = (np.log10(subhalo_mass*1e10/0.6774) > 9.05)

    ma9_index  = condition1 & (np.log10(subhalo_mass*1e10/0.6774) > 9.) & (np.log10(subhalo_mass*1e10/0.6774) < 10.)
    ma10_index = condition1 & (np.log10(subhalo_mass*1e10/0.6774) > 10.) & (np.log10(subhalo_mass*1e10/0.6774) < 11.)
    ma11_index = condition1 & (np.log10(subhalo_mass*1e10/0.6774) > 11.) & (np.log10(subhalo_mass*1e10/0.6774) < 12.)
    ma12_index = condition1 & (np.log10(subhalo_mass*1e10/0.6774) > 12.) & (np.log10(subhalo_mass*1e10/0.6774) < 13.)
    ma13_index = condition1 & (np.log10(subhalo_mass*1e10/0.6774) > 13.)
    
    all_mass_ind=[ma9_index,ma10_index,ma11_index,ma12_index,ma13_index]
    
    cond_all = condition1 & condition2
    
    #the radius projected in x-axis
    #subhalo_rx= np.zeros(len(subhalo_posx[cond_all]))
    #subhalo_rx = np.sqrt(4*subhalo_radhm[cond_all]**2 - (subhalo_posy[cond_all] - y_pos)**2)
    #subhalo_dwl = ((1+z_halo[cond_all])*subhalo_vmax[cond_all]/c )* lya
    #label all pixels in the halo range

    #subhalo_rx= np.zeros(len(subhalo_posx[cond_all]))
    subhalo_rx_sq = (4*subhalo_radhm**2 - (subhalo_posy - y_pos)**2)
    subhalo_dwl = ((1+z_halo)*subhalo_vmax/c )* lya
    
    # Assuming f, subhalo_posx, subhalo_posy, subhalo_radhm, subhalo_vz, subhalo_vmax, z_halo, lya, c, and ray_z are already defined
    LOS_xy = np.array((f['ray_pos'][:, 0], f['ray_pos'][:, 1])).T
    subhalo_positions = np.array((subhalo_posx[condition2], subhalo_posy[condition2])).T
    subhalo_radii = 2 * subhalo_radhm[condition2]
    z_halo_cond = z_halo[condition2]
    subhalo_vz_cond =subhalo_vz[condition2]
    subhalo_vmax_cond = subhalo_vmax[condition2]
    
    # Create a KDTree for LOS_xy
    tree = cKDTree(LOS_xy)
    
    # Initialize lists to store results
    los_halo_ind = []
    wl_halo_ind_all = []
    
    # Create the LOS_MASK array
    LOS_MASK = np.zeros((len(f['ray_pos']), len(ray_z)), dtype=np.float16)
    
    # Iterate over each halo
    for i, (pos, radius) in enumerate(zip(subhalo_positions, subhalo_radii)):
        # Find all LOS within the radius in the x-y plane
        los_indices0 = tree.query_ball_point(pos, radius)
        
        # Store the results
        los_halo_ind.append(los_indices0)
        
        # Calculate the wavelength range for the halo in the z-direction
        c_z_i = (z_halo_cond[i]+1)
        halo_dwl_min = (1 + z_halo_cond[i] + c_z_i*subhalo_vz_cond[i] / c - c_z_i*subhalo_vmax_cond[i] / c) * lya
        halo_dwl_max = (1 + z_halo_cond[i] + c_z_i*subhalo_vz_cond[i] / c + c_z_i*subhalo_vmax_cond[i] / c) * lya
        
        # Find the wavelength indices that are covered by the halo
        wl_indices0 = np.where((ray_z >= halo_dwl_min) & (ray_z <= halo_dwl_max))[0]
        
        #if i <= 30:
        #    print(z_halo_cond[i],halo_dwl_min,halo_dwl_max,2*subhalo_radhm[condition2][i],len(wl_indices0), len(los_indices0),los_indices0[0:10])
        
        # the halo wl for each los that intersect with halo i     
        wl_los_halo_indices = []
        for j in los_indices0:
            # Store the results
            wl_los_halo_indices.append(wl_indices0)
            LOS_MASK[j, wl_indices0] = 1
        
        wl_halo_ind_all.append(wl_los_halo_indices)
    
    # los_indices now contains the indices of LOS in the x-y plane for each halo
    # wavelength_indices contains the wavelength indices covered by each halo in the z-direction
    # only use the los around the halo
    
    #the plot range in unit of virial radius
    r_f_x = 6
    
    r_f_y = 2
    
    #los that go through the halo
    x_ind_plot= \
    np.where((f['ray_pos'][y_ind_not_DLA][:, 0] >= halo_0_x - r_f_x*halo_0_r) & (f['ray_pos'][y_ind_not_DLA][:, 0] <= halo_0_x + r_f_x*halo_0_r))[0]
    
    #flux in wl range
    #flux_lya = f['flux'][:, wl_ind]
    #flux_map= flux_lya[y_ind_not_DLA]
    
    flux_map = f['flux'][y_ind_not_DLA[x_ind_plot]][:, wl_ind]
    
    #remove los with repeated x
    ind_unique = np.unique(f['ray_pos'][:,0][y_ind_not_DLA[x_ind_plot]], return_index=True)
    x_pos_unique = f['ray_pos'][:,0][y_ind_not_DLA[x_ind_plot]][ind_unique[1]]
    
    flux_map_unique = flux_map[ind_unique[1]]
    
    ind_sort_unique = np.argsort(x_pos_unique)
    x_unique_sort = x_pos_unique[ind_sort_unique]
    
    #only plot the halos with more than 10 los
    Mini_LOS = 10
    skip1 = len(x_unique_sort) <= Mini_LOS
    skip2 = (ray_z[0]> wl_halo_0-r_f_y*halo_0_dwl)| (ray_z[-1]< wl_halo_0+r_f_y*halo_0_dwl)
    skip3 = ((halo_0_x - r_f_x*halo_0_r) <0) | ((halo_0_x + r_f_x*halo_0_r) >Lbox*1000)
    
    if skip1|skip2|skip3:
        print(f"n(LOS) <= 10 {skip1}, y out or range {skip2}, x out or range {skip3} , skip")
        
        continue
    
    flux_map_unique_sort = flux_map_unique[ind_sort_unique]
    
    #map0=np.array(np.vstack(flux_map_unique))
    map_fine=np.array(np.vstack(flux_map_unique_sort))
    
    #interpolate the color map to finer grid
    f_map_fine = RegularGridInterpolator((x_unique_sort, ray_z), map_fine, method='cubic')
    
    # interpolation resolution in x ~4 kpc,  original resolution (~8.75 kpc, for 2d slices with width ~35kpc)
    fine_res = 4#km/s
    x_grid_fine = np.linspace(x_unique_sort[0],x_unique_sort[-1],np.int32((x_unique_sort[-1]-x_unique_sort[0])/fine_res)) 
    X_plot_fine, Y_plot_fine = np.meshgrid(x_grid_fine[0:-1], ray_z[0:-1], indexing='ij')
    map_plot_fine = f_map_fine((X_plot_fine, Y_plot_fine))
    d_arr_z_fine= np.arange(len(ray_z)) * dl *1000 # in kpc/h
    
    
    gridspec = {'width_ratios': [1,0.07,1,1,1, 0.07]}
    fig3, axes3 = plt.subplots(nrows=1, ncols=6, figsize=(20,10),gridspec_kw=gridspec)
    
    #axes2[0].scatter(subhalo_posx[cond_all],wl_halo[cond_all], marker='*', s=2, c='red')
    c2 = axes3[0].pcolormesh(x_grid_fine,ray_z , map_plot_fine.T, cmap='Greys', vmin=0., vmax=1.,zorder = 0)
    
    #color_arr= ['green','olive','yellow','orange','red']
    color_arr= ['red','red','red','red','red']
    for i_mass in np.arange(0,len(all_mass_ind)):
        axes3[0].errorbar(subhalo_posx[all_mass_ind[i_mass]],wl_halo[all_mass_ind[i_mass]],
                  xerr=np.sqrt(subhalo_rx_sq[all_mass_ind[i_mass]]),yerr=subhalo_dwl[all_mass_ind[i_mass]], 
                      fmt='.', linewidth=1, capsize=1, c=color_arr[i_mass])
    
    for x_i in x_los_DLA:
        axes3[0].axvline(x=x_i, ls=':', color='green', lw=1.0, alpha=0.5,label = 'DLA',zorder =1)
    
    for x_i in x_halo0_los:
        axes3[0].axvline(x=x_i, ls='-', color='yellow', lw=1.0, alpha=0.5,label = 'LOS',zorder =1)
    
    
    #ax3.set_yticks(d_arr_z)
    #plot mass=12 halo
    #axes2[0].scatter(np.atleast_1d(halo_0_x),np.atleast_1d(wl_halo_0_wl), marker='*', s=1, c='green')
    
    axes3[0].set_title('flux')
    # set the limits of the plot to the limits of the data
    plt.colorbar(c2, cax=axes3[1])
    
    #plot range in y-axis, in unit of the r_vmax
    r_f_y = 2
    z_ind_plot_fine = np.where((ray_z > wl_halo_0-r_f_y*halo_0_dwl) 
                           & (ray_z < wl_halo_0+r_f_y*halo_0_dwl))
    
    ray_z_fineplot = ray_z[z_ind_plot_fine] 
    vel_plot_fine = (ray_z_fineplot-(ray_z_fineplot[0]+ray_z_fineplot[-1])/2)/wl_halo_0 * c # in km/s
    
    #z_ind_plot_rebin = np.where((wl_rebin > wl_halo_0-r_f_y*halo_0_dwl) 
    #                  & (wl_rebin < wl_halo_0+r_f_y*halo_0_dwl))
    
    #ray_z_plot2 = wl_rebin[z_ind_plot_rebin] 
    #vel_plot = (ray_z_plot2-(ray_z_plot2[0]+ray_z_plot2[-1])/2)/wl_halo_0 * c # in km/s
    
    ax3 = axes3[0].twinx()
    ax3.yaxis.set_label_position("right")
    ax3.yaxis.tick_right()
    ax3.set_yticks(vel_plot_fine)
    ax3.set_ylabel(r'$\Delta v$ km/s',fontsize=14)
    
    N_y_ticks = np.ceil((vel_plot_fine[-1]-vel_plot_fine[0])/200)+2
    ax3.yaxis.set_major_locator(MaxNLocator(N_y_ticks))
    ax3.yaxis.set_minor_locator(AutoMinorLocator())
    
    axes3[0].set_xlim( halo_0_x-r_f_x*halo_0_r , halo_0_x + r_f_x*halo_0_r)
                  
    N_x_ticks = np.ceil(2*r_f_x*halo_0_r/1000)
    axes3[0].xaxis.set_major_locator(MaxNLocator(N_x_ticks))
    
    axes3[0].set_ylim(wl_halo_0-r_f_y*halo_0_dwl, wl_halo_0+r_f_y*halo_0_dwl)
    #axes3[0].set_ylim(d_arr_z[z_ind_plot][0],d_arr_z[z_ind_plot][-1])
    axes3[0].yaxis.set_major_locator(MaxNLocator(N_y_ticks))
    axes3[0].xaxis.set_minor_locator(AutoMinorLocator())
    axes3[0].yaxis.set_minor_locator(AutoMinorLocator())
    axes3[0].set_yticks([])
    #embed()
    #axes3[0].set_xlabel(r'X [ckpc/h]',fontsize=14)
    
    #dl_arr_plot = d_arr_z_rebin[z_ind_plot_rebin]-(d_arr_z_rebin[z_ind_plot_rebin][0]+d_arr_z_rebin[z_ind_plot_rebin][-1])/2
    
    dl_arr_fine = d_arr_z_fine[z_ind_plot_fine]-(d_arr_z_fine[z_ind_plot_fine][0]+d_arr_z_fine[z_ind_plot_fine][-1])/2
    
    ax4 = axes3[0].twinx()
    ax4.yaxis.set_label_position("left")
    ax4.yaxis.tick_left()
    ax4.set_yticks(np.int32(dl_arr_fine))
    ax4.set_ylabel(r'Y [ckpc/h]',fontsize=14)
    #ax4.set_ylim(np.int32(d_arr_z[z_ind_plot])[0],np.int32(d_arr_z[z_ind_plot])[1])
    
    N_z_dl = np.ceil((d_arr_z_fine[z_ind_plot_fine][-1] - d_arr_z_fine[z_ind_plot_fine][0])/1000)
    ax4.yaxis.set_major_locator(MaxNLocator(N_z_dl))
    ax4.yaxis.set_minor_locator(AutoMinorLocator())
    
    axes3[0].set_xticks([])
    ax5 = axes3[0].twiny()
    ax5.tick_params(top=False, labeltop=False, bottom=True, labelbottom=True)
    x_axis_plot = np.linspace( -r_f_x*halo_0_r, r_f_x*halo_0_r, len(x_grid_fine))
    ax5.set_xticks(np.int32(x_axis_plot))
    N_x_grid = 3
    ax5.xaxis.set_major_locator(MaxNLocator(N_x_grid))
    ax5.set_xlabel(r'X [ckpc/h]',fontsize=14)
    ax5.xaxis.set_label_position('bottom') 
    ax5.xaxis.set_minor_locator(AutoMinorLocator())
    ax5.xaxis.tick_bottom()

    rebin_size = [1,40]

    # rebin plots
    for i_plot in np.arange(2,4):
        
        wl_bin_i = rebin_size[i_plot-2]
        ######rebinned plot
        # rebin to new wavelength array
        wl_rebin = wave_rebin(wl_bin_i, ray_z)[0]
        rebin_res_i = wave_rebin(wl_bin_i, ray_z)[1]
        #print("res:", rebin_res_i ," km/s")
        
        flux_rebin =  spectres.spectres(wl_rebin, ray_z, flux_map_unique_sort, fill=True, verbose=True)
        d_arr_z_rebin= np.arange(len(wl_rebin)) * dl *1000 * len(ray_z)/len(wl_rebin)# in kpc/h
        
        #map0=np.array(np.vstack(flux_map_unique))
        map0=np.array(np.vstack(flux_rebin))
        
        #interpolate the color map to finer grid
        f_map = RegularGridInterpolator((x_unique_sort, wl_rebin), map0, method='cubic')
        
       # interpolation resolution in x ~4 kpc,  original resolution (~8.75 kpc, for 2d slices with width ~35kpc)
        interpol_res = 4#km/s
        x_grid = np.linspace(x_unique_sort[0],x_unique_sort[-1],np.int32((x_unique_sort[-1]-x_unique_sort[0])/interpol_res)) 
        X_plot, Y_plot = np.meshgrid(x_grid[0:-1], wl_rebin[0:-1], indexing='ij')
        
        map_plot = f_map((X_plot,Y_plot))

        c2 = axes3[i_plot].pcolormesh(x_grid,wl_rebin , map_plot.T, cmap='Blues', vmin=0., vmax=0.25)
        
        #axes2[0].scatter(subhalo_posx[cond_all],wl_halo[cond_all], marker='*', s=2, c='red')
        for i_mass in np.arange(0,len(all_mass_ind)):
            axes3[i_plot].errorbar(subhalo_posx[all_mass_ind[i_mass]],wl_halo[all_mass_ind[i_mass]],
                  xerr=(np.sqrt(subhalo_rx_sq[all_mass_ind[i_mass]])),yerr=subhalo_dwl[all_mass_ind[i_mass]], 
                      fmt='.', linewidth=1, capsize=1, c=color_arr[i_mass])
        
        #axes3[i_plot].set_title(f'rebin {np.int16(rebin_res_i):d} km/s')
        axes3[i_plot].annotate(f'Res: /n{np.int16(rebin_res_i):.1f} km/s', xy=(0.1, 0.95), xycoords='axes fraction',color = 'lightgrey',fontsize=12)

        
        #plot color bar only once
        #if i_plot ==2:
        #    plt.colorbar(c2, cax=axes3[-1])
    
        # set the limits of the plot to the limits of the data
        #axes3[2].set_ylabel(r'wavelength',fontsize=14)
        axes3[i_plot].set_xlim( halo_0_x-r_f_x*halo_0_r, halo_0_x + r_f_x*halo_0_r)
        axes3[i_plot].xaxis.set_minor_locator(AutoMinorLocator())
        axes3[i_plot].yaxis.set_minor_locator(AutoMinorLocator())
        axes3[i_plot].xaxis.set_major_locator(MaxNLocator(N_x_ticks))
        axes3[i_plot].set_ylim(wl_halo_0-r_f_y*halo_0_dwl,wl_halo_0+r_f_y*halo_0_dwl)
        #axes3[0].set_ylim(d_arr_z[z_ind_plot][0],d_arr_z[z_ind_plot][-1])
        axes3[i_plot].yaxis.set_major_locator(MaxNLocator(N_y_ticks))
        axes3[i_plot].set_yticks([])
        #embed()
        #axes3[2].set_xlabel(r'X [ckpc/h]',fontsize=14)
    
        ax32 = axes3[i_plot].twinx()
        ax32.yaxis.set_label_position("right")
        ax32.yaxis.tick_right()
        ax32.set_yticks(vel_plot_fine)
        ax32.set_ylabel(r'$\Delta v$ km/s',fontsize=10,loc="top")
        ax32.yaxis.set_major_locator(MaxNLocator(N_y_ticks))
        ax32.yaxis.set_minor_locator(AutoMinorLocator())
        
        ax42 = axes3[i_plot].twinx()
        ax42.yaxis.set_label_position("left")
        ax42.yaxis.tick_left()
        ax42.set_yticks(np.int32(dl_arr_fine))
        ax42.set_ylabel(r'Y [ckpc/h]',fontsize=10,loc="bottom")
        #ax4.set_ylim(np.int32(d_arr_z[z_ind_plot])[0],np.int32(d_arr_z[z_ind_plot])[1])
        ax42.yaxis.set_major_locator(MaxNLocator(N_z_dl))
        ax42.yaxis.set_minor_locator(AutoMinorLocator())
        
        axes3[i_plot].set_xticks([])
        ax52 = axes3[i_plot].twiny()
        ax52.tick_params(top=False, labeltop=False, bottom=True, labelbottom=True)
        ax52.set_xticks(np.int32(x_axis_plot))
        ax52.xaxis.set_major_locator(MaxNLocator(N_x_grid))
        ax52.set_xlabel(r'X [ckpc/h]',fontsize=10)
        ax52.xaxis.set_label_position('bottom') 
        ax52.xaxis.set_minor_locator(AutoMinorLocator())
        ax52.xaxis.tick_bottom()

    
    smooth_bin = rebin_size[-1]
    smoothed_flux = convolve1d(flux_map_unique_sort, [1/smooth_bin]*smooth_bin ,axis=0)
    
    #map0=np.array(np.vstack(flux_map_unique))
    map0=np.array(np.vstack(smoothed_flux))
    
    #interpolate the color map to finer grid
    f_map = RegularGridInterpolator((x_unique_sort, ray_z), map0, method='cubic')
    
   # interpolation resolution in x ~4 kpc,  original resolution (~8.75 kpc, for 2d slices with width ~35kpc)
    interpol_res = 4#km/s
    x_grid = np.linspace(x_unique_sort[0],x_unique_sort[-1],np.int32((x_unique_sort[-1]-x_unique_sort[0])/interpol_res)) 
    X_plot, Y_plot = np.meshgrid(x_grid[0:-1], ray_z[0:-1], indexing='ij')
    
    map_plot = f_map((X_plot,Y_plot))

    c0 = axes3[-2].pcolormesh(x_grid,ray_z , map_plot.T, cmap='Blues', vmin=0., vmax=0.25)
    
    #axes2[0].scatter(subhalo_posx[cond_all],wl_halo[cond_all], marker='*', s=2, c='red')
    for i_mass in np.arange(0,len(all_mass_ind)):
        axes3[-2].errorbar(subhalo_posx[all_mass_ind[i_mass]],wl_halo[all_mass_ind[i_mass]],
              xerr=np.sqrt(subhalo_rx_sq[all_mass_ind[i_mass]]),yerr=subhalo_dwl[all_mass_ind[i_mass]], 
                  fmt='.', linewidth=1, capsize=1, c=color_arr[i_mass])
    
    #axes3[-2].set_title(f'smoothing {np.int16(smooth_bin):d} km/s')
    axes3[-2].annotate(f'Smoothing: /n{np.int16(wave_rebin(smooth_bin, ray_z)[1]):.1f} km/s', xy=(0.05, 0.95), xycoords='axes fraction',color = 'lightgrey',fontsize=12)
    
    #plot color bar only once
    plt.colorbar(c0, cax=axes3[-1])

    # set the limits of the plot to the limits of the data
    #axes3[2].set_ylabel(r'wavelength',fontsize=14)
    axes3[-2].set_xlim( halo_0_x-r_f_x*halo_0_r, halo_0_x + r_f_x*halo_0_r)
    axes3[-2].xaxis.set_minor_locator(AutoMinorLocator())
    axes3[-2].yaxis.set_minor_locator(AutoMinorLocator())
    axes3[-2].xaxis.set_major_locator(MaxNLocator(N_x_ticks))
    axes3[-2].set_ylim(wl_halo_0-r_f_y*halo_0_dwl,wl_halo_0+r_f_y*halo_0_dwl)
    #axes3[2].set_ylim(d_arr_z[z_ind_plot][0],d_arr_z[z_ind_plot][-1])
    axes3[-2].yaxis.set_major_locator(MaxNLocator(N_y_ticks))
    axes3[-2].set_yticks([])
    #embed()
    #axes3[2].set_xlabel(r'X [ckpc/h]',fontsize=14)

    ax32 = axes3[-2].twinx()
    ax32.yaxis.set_label_position("right")
    ax32.yaxis.tick_right()
    ax32.set_yticks(vel_plot_fine)
    ax32.set_ylabel(r'$\Delta v$ km/s',fontsize=10,loc="top")
    ax32.yaxis.set_major_locator(MaxNLocator(N_y_ticks))
    ax32.yaxis.set_minor_locator(AutoMinorLocator())
    
    ax42 = axes3[-2].twinx()
    ax42.yaxis.set_label_position("left")
    ax42.yaxis.tick_left()
    ax42.set_yticks(np.int32(dl_arr_fine))
    ax42.set_ylabel(r'Y [ckpc/h]',fontsize=10,loc="bottom")
    #ax4.set_ylim(np.int32(d_arr_z[z_ind_plot])[0],np.int32(d_arr_z[z_ind_plot])[1])
    ax42.yaxis.set_major_locator(MaxNLocator(N_z_dl))
    ax42.yaxis.set_minor_locator(AutoMinorLocator())
    
    axes3[-2].set_xticks([])
    ax52 = axes3[-2].twiny()
    ax52.tick_params(top=False, labeltop=False, bottom=True, labelbottom=True)
    ax52.set_xticks(np.int32(x_axis_plot))
    ax52.xaxis.set_major_locator(MaxNLocator(N_x_grid))
    ax52.set_xlabel(r'X [ckpc/h]',fontsize=10)
    ax52.xaxis.set_label_position('bottom') 
    ax52.xaxis.set_minor_locator(AutoMinorLocator())
    ax52.xaxis.tick_bottom()
    
    #embed()
    #fig3.legend(loc='upper right')
    #fig3.tight_layout()
    plt.subplots_adjust(wspace=2.0, hspace=0)
    plt.show()
    #savename2 =('z2_zoomin_halos_y'+str(i_halo)+'_mass11.5.pdf')
    #plt.savefig(savename2, dpi=100)
    plt.close()

halo  4
n(LOS) <= 10 False, y out or range True, x out or range False , skip
halo  10
